In [ ]:
# %% [code]
# %% [code]
# %% [code]
# %% [code]
#!/usr/bin/env python
# coding: utf-8

# In[76]:


from pathlib import Path
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn


# In[77]:


df_train = pd.read_parquet('/kaggle/input/mdsb-2023/train.parquet')


# In[78]:


import holidays
def is_fr_bank_holiday(date):
    if date in holidays.France():
        return 1
    else:
        return 0
    
def is_weekend(day):
    if day == 6 or day == 7:
        return 1
    else:
        return 0


def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour
    
    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])


# In[79]:


def _merge_external_data(X):
    file_path = '/kaggle/input/mdsb-2023/external_data.csv'
    df_ext = pd.read_csv(file_path, parse_dates=["date"])
    
    X['date'] = pd.to_datetime(X['date']).astype('datetime64[ns]')
    df_ext['date'] = pd.to_datetime(df_ext['date']).astype('datetime64[ns]')
    
    X = X.copy()
    # When using merge_asof left frame need to be sorted
    X["orig_index"] = np.arange(X.shape[0])
    X = pd.merge_asof(
        X.sort_values("date"), df_ext[["date", "t", 'pres', 'raf10', 'u', 'ff', 'vv', 'tend24', 'cod_tend', 'rr3', 'rr12']].sort_values("date"), on="date"
    )
    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]
    return X


# In[80]:


df_train = _merge_external_data(df_train)


# In[81]:


df_train['counter_age_days'] = df_train['date'] - df_train['counter_installation_date']
df_train['counter_age_days'] = df_train['counter_age_days'].dt.days


# In[82]:


X_dates_encoding = _encode_dates(df_train[["date"]])
df_train = pd.concat([df_train, X_dates_encoding], axis=1)
df_train = df_train.dropna(axis=0)

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoder.fit(df_train[["counter_name"]])
df_train["counter_name"] = encoder.transform(df_train[["counter_name"]])


# In[83]:


selected_features = ['latitude', 't', 'u', 'month', 'weekday', 'hour', 'day', "counter_name"]
y_train = df_train['log_bike_count']
X_train = df_train[selected_features]


# In[ ]:


from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# Assuming df is your DataFrame containing the required columns
# Replace 'df' with the actual variable containing your dataset


# Initialize and train a Gradient Boosting Regressor
pipe = XGBRegressor(learning_rate=0.22, max_depth=6, n_estimators=600, gamma=0.01, min_child_weight=7, reg_alpha=0.15)
pipe.fit(X_train, y_train)

#max_features='sqrt', min_samples_leaf=2, min_samples_split=2, subsample=0.75
# In[70]:


df_test = pd.read_parquet('/kaggle/input/mdsb-2023/final_test.parquet')


# In[71]:


df_test = _merge_external_data(df_test)


# In[72]:


test_dates_encoding = _encode_dates(df_test[["date"]])
df_test = pd.concat([df_test, test_dates_encoding], axis=1) 


# In[73]:


df_test['counter_age_days'] = df_test['date'] - df_test['counter_installation_date']
df_test['counter_age_days'] = df_test['counter_age_days'].dt.days

df_test["counter_name"] = encoder.transform(df_test[["counter_name"]])

# In[75]:


y_pred = pipe.predict(df_test[selected_features])
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission.csv", index=False)


# In[ ]:



